# Project Title
* Movie Market Analysis

# Project Overview

For this project, you will use exploratory data analysis to generate insights for a business stakeholder.

Business Problem
* Your company now sees all the big companies creating original video content and they want to get in on the fun. They have decided to create a new movie studio, but they don’t know anything about creating movies. You are charged with exploring what types of films are currently doing the best at the box office. You must then translate those findings into actionable insights that the head of your company's new movie studio can use to help decide what type of films to create.

# Goals
  
The objective is to identify which types of movies perform best commercially and critically, and to translate these insights into clear business recommendations on what kinds of films the studio should produce.


## Data loading and initial inspection

Multiple datasets are used to capture box office performance, production budgets, genres, and audience and critic ratings.  
Before analysis, we inspect the structure and completeness of each dataset.


In [ ]:
# Loading the libraries

import pandas as pd
import numpy as numpy
import matplotlib.pyplot as pyplot
import sqlite3

In [ ]:
# loading the csv files
movie_gross = pd.read_csv('../zippedData/bom.movie_gross.csv.gz')
movies = pd.read_csv('../zippedData/tmdb.movies.csv.gz')
movie_budgets = pd.read_csv('../zippedData/tn.movie_budgets.csv.gz')

# loading the tsv files
movie_info = pd.read_csv('../zippedData/rt.movie_info.tsv.gz', sep='\t')
reviews = pd.read_csv('../zippedData/rt.reviews.tsv.gz',sep='\t',encoding='latin1')


In [ ]:
reviews.info()

In [ ]:
movies.info()

In [ ]:
movie_info.info()

In [ ]:
movie_gross.info()

In [ ]:
movie_budgets.info()

In [ ]:
movie_budgets

In [ ]:
# converting tsv files to csv  files 
movie_info.to_csv('rt.movie_info.csv',index=False)

reviews.to_csv('rt.reviews.csv',index=False)

In [ ]:
# unzipping the database
import zipfile

with zipfile.ZipFile('../zippedData/im.db.zip','r') as zip_ref:
    zip_ref.extractall('zippedData')

In [ ]:
# creating a database connection
conn = sqlite3.connect('im.db')
cursor = conn.cursor()

In [ ]:
# importing the IMDB database

imdb_tables = """
    SELECT *
    FROM sqlite_master 
    WHERE type = 'table';
    """

pd.read_sql(imdb_tables,conn)

## Methods

## Results

### Business Recommendation 1

### Business Recommendation 2

### Business Recommendation 3